# Models

The PCA shapes for 79 models is at: `../models/cars/2_watertight/mc`

The 79 watertight models is at: `../models/cars/2_watertight/watertight`

# Codes, PCA basis and shape bias files

The codes for the 79 models in the 10 dimensional PCA space is at: `./codes.npy`

The basis to map from the 10 dimensional PCA space to the 3E2786 dimensional TSDF space is at: `./basis.npy`

The shape bias file is at: `./bias.npy`

# Load TSDF for 79 models and acquire codes and basis of the PCA model

In [87]:
import os
import h5py
import argparse
import numpy as np
import car_models
import scipy.io

resolution = 64

data_path = './data/one_example_64/'
tsdf_path = data_path + '4_tsdf'
tsdf_list = []
mat_path = data_path + '1_s_t'
mat_list = []
for idx, model in enumerate(car_models.models):
    print model
    tsdf_file = os.path.join(tsdf_path, model.name + '.npy')
    tsdf = np.load(tsdf_file)
    print tsdf.shape
    mat_file = os.path.join(mat_path, model.name + '.mat')
    mat = scipy.io.loadmat(mat_file)
    tsdf_list.append(tsdf.reshape((1, -1)))
    mat_list.append(mat)
    
tsdf_all = np.concatenate(tsdf_list)
print tsdf_all.shape

Label(name='baojun-310-2017', id=0, category='2x', categoryId=0)
(64, 64, 64)
Label(name='biaozhi-3008', id=1, category='2x', categoryId=0)
(64, 64, 64)
Label(name='biaozhi-liangxiang', id=2, category='2x', categoryId=0)
(64, 64, 64)
Label(name='bieke-yinglang-XT', id=3, category='2x', categoryId=0)
(64, 64, 64)
Label(name='biyadi-2x-F0', id=4, category='2x', categoryId=0)
(64, 64, 64)
Label(name='changanbenben', id=5, category='2x', categoryId=0)
(64, 64, 64)
Label(name='dongfeng-DS5', id=6, category='2x', categoryId=0)
(64, 64, 64)
Label(name='feiyate', id=7, category='2x', categoryId=0)
(64, 64, 64)
Label(name='fengtian-liangxiang', id=8, category='2x', categoryId=0)
(64, 64, 64)
Label(name='fengtian-MPV', id=9, category='2x', categoryId=0)
(64, 64, 64)
Label(name='jilixiongmao-2015', id=10, category='2x', categoryId=0)
(64, 64, 64)
Label(name='lingmu-aotuo-2009', id=11, category='2x', categoryId=0)
(64, 64, 64)
Label(name='lingmu-swift', id=12, category='2x', categoryId=0)
(64, 64,

In [88]:
# from hdf5_utils import *

# cars_hdf5_name = '../models/cars/2_watertight/sdf.h5'

# h5 = read_hdf5(cars_hdf5_name)
# S = h5.reshape((h5.shape[0], -1))

S_train = tsdf_all[:, :]

S_mean = np.mean(S_train, axis=0)
S_white_train = S_train - S_mean

from sklearn import decomposition
pca = decomposition.PCA(n_components=10)
pca.fit(S_white_train)
V = pca.components_.T # basis (mapping from high to low dim feature space), 32786*10
print V.shape

beta = np.dot(V.T, S_white_train.T) # embedding for 79 models, 10*79
print beta.shape

np.save(data_path+'basis', V)
np.save(data_path+'codes', beta)
np.save(data_path+'bias', S_mean)

print np.amax(beta, axis=1)
print np.amin(beta, axis=1)
print np.var(beta, axis=1)

(262144, 10)
(10, 79)
[11.509787   5.9022603  7.2110186  4.5671206  4.4800825  2.0218406
  3.6571462  2.2625911  2.5543284  1.3440583]
[-8.08274   -2.8706794 -1.9511607 -2.5483506 -2.2561579 -1.3636198
 -2.0216494 -1.612346  -1.5834291 -1.281826 ]
[23.161276    2.977005    1.7910368   1.2807842   1.1888341   0.573231
  0.54731107  0.480438    0.33902985  0.2877932 ]


# Recover the shape from the basis and code for one model

In [89]:
# Demo for compressing and recovering the 1st model.
# Assuming beta_input is the input PCA code with dimension [1, 10]
beta_input = beta[:, :]
S_gnd = S_train[:, :]
S_gnd = S_gnd.reshape((-1, resolution, resolution, resolution))

S_recover = np.dot(V, beta_input).T+S_mean.reshape((1, -1))
print S_recover.shape
S_recover = S_recover.reshape((-1, resolution, resolution, resolution))

import librender
import libmcubes
from skimage import measure
import pickle
import scipy.io

for idx in range(S_recover.shape[0]):
    print idx
#     vertices, faces, normals, values = measure.marching_cubes_lewiner(S_recover[idx].transpose(1, 0, 2), 0)
    vertices, faces = libmcubes.marching_cubes(S_recover[idx], 0)
    vertices /= resolution
    vertices -= 0.5

    # from mesh_utils import Mesh
    # mesh = Mesh(vertices, faces)
    # mesh.to_off('a.off')

    libmcubes.export_off(vertices, faces, data_path + '5_recon/' + '%d.off'%idx)
    
for idx in range(S_gnd.shape[0]):
    print idx
    vertices, faces = libmcubes.marching_cubes(S_gnd[idx], 0)
#     vertices, faces, _, _ = measure.marching_cubes_lewiner(S_gnd[idx], 0)
    vertices /= resolution
    vertices -= 0.5
    
    mat = mat_list[idx]
    scales_ori = 1./mat['scales'][0][0]

    libmcubes.export_off(vertices*scales_ori, faces, data_path + '5_gt/' + '%s.off'%(car_models.models[idx].name))
    
    mydict = {'vertices': vertices*scales_ori, 'faces': faces+1}
    output = open(data_path + '5_gt/' + '%s.pkl'%(car_models.models[idx].name), 'wb')
    pickle.dump(mydict, output)
    output.close()
    
    scipy.io.savemat(data_path + '5_gt/' + '%s.mat'%(car_models.models[idx].name), 
                    mydict)

(79, 262144)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78


In [58]:
import pickle as pkl
with open('/Users/ruizhu/Documents/019-SUV.pkl', 'r') as f:
    model = pkl.load(f)
print model

print model['vertices'].shape, model['faces'].shape

print np.max(model['faces']), np.min(model['faces'])
libmcubes.export_off(model['vertices'], model['faces']-1, '/Users/ruizhu/Documents/019-SUV.pkl.off')
    

{'car_type': 'SUV', 'vertices': array([[ 1.06786262, -0.33273001,  0.51763339],
       [ 1.06286721, -0.27874485,  0.51430087],
       [ 1.06278939, -0.27568073,  0.51322238],
       ...,
       [-1.06286706, -0.27874485,  0.51429904],
       [-1.06484504, -0.33332745,  0.51697788],
       [-1.06786262, -0.33272984,  0.51763217]]), 'faces': array([[3755, 3754, 3924],
       [3884, 4241, 4096],
       [3924, 4241, 3755],
       ...,
       [1800, 1798, 1804],
       [1799, 1800, 1804],
       [1722, 1802, 1798]], dtype=uint16)}
(4294, 3) (5000, 3)
4294 1
